### This part processed the sensor data: 
- transfered CAL_DATE to Date(as dt)
- add DOW, 
- add total spots & occupied/unoccupied/unknown ratios for different categories.
- processed data is saved seperately so that we can load in later when needed.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)

In [4]:
sensor_dt = pd.read_csv('sensordt.csv', parse_dates= [12], infer_datetime_format=True)
col_dt = sensor_dt['Date'].copy()
del sensor_dt

In [5]:
sensor = pd.read_csv('ParkingSensorData.csv', parse_dates= True, infer_datetime_format=True)
sensor.drop(sensor.tail(1).index, inplace=True)

In [6]:
sensor['Date'] = col_dt
sensor['DOW'] = sensor['Date'].dt.weekday

In [7]:
def cal_spot(column, df):
    df[column + "_spots"] = df[column]/3600
    
cols = ['TOTAL_TIME', 'OP_TIME', 'NONOP_TIME', 'GMP_TIME', 'COMM_TIME']
for col in cols:
    cal_spot(col, sensor)

In [8]:
def cal_pct(cols, total, df):
    for col in cols:
        df[col+"_pct"] = df[col]/df[total]

pct_cols = [(['TOTAL_OCCUPIED_TIME', 'TOTAL_VACANT_TIME', 'TOTAL_UNKNOWN_TIME'],'TOTAL_TIME'),
            (['OP_OCCUPIED_TIME', 'OP_VACANT_TIME', 'OP_UNKNOWN_TIME'],'OP_TIME'),
            (['NONOP_OCCUPIED_TIME', 'NONOP_VACANT_TIME','NONOP_UNKNOWN_TIME'],'NONOP_TIME'),
            (['GMP_OCCUPIED_TIME','GMP_VACANT_TIME', 'GMP_UNKNOWN_TIME'],'GMP_TIME'),
            (['COMM_OCCUPIED_TIME', 'COMM_VACANT_TIME', 'COMM_UNKNOWN_TIME'],'COMM_TIME')]

for pair in pct_cols:
    cal_pct(pair[0],pair[1],sensor)

In [9]:
sensor.head()

,BLOCK_ID,STREET_NAME,BLOCK_NUM,STREET_BLOCK,AREA_TYPE,PM_DISTRICT_NAME,RATE,START_TIME_DT,TOTAL_TIME,TOTAL_OCCUPIED_TIME,...,OP_UNKNOWN_TIME_pct,NONOP_OCCUPIED_TIME_pct,NONOP_VACANT_TIME_pct,NONOP_UNKNOWN_TIME_pct,GMP_OCCUPIED_TIME_pct,GMP_VACANT_TIME_pct,GMP_UNKNOWN_TIME_pct,COMM_OCCUPIED_TIME_pct,COMM_VACANT_TIME_pct,COMM_UNKNOWN_TIME_pct
0,41522,FILLMORE ST,22.0,FILLMORE ST 2200,Pilot,Fillmore,NaN,01-jul-2012 21:00:00,61200.0,35847.0,...,NaN,0.585735,0.414265,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,41524,FILLMORE ST,24.0,FILLMORE ST 2400,Pilot,Fillmore,NaN,01-jul-2012 22:00:00,64800.0,27576.0,...,NaN,0.425556,0.574444,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,50001,JACKSON ST,1.0,JACKSON ST 100,Pilot,Downtown,NaN,01-jul-2012 02:00:00,79200.0,39985.0,...,NaN,0.504861,0.495139,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,50002,JACKSON ST,2.0,JACKSON ST 200,Pilot,Downtown,NaN,01-jul-2012 08:00:00,82800.0,30465.0,...,NaN,0.367935,0.632065,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,10903,09TH AVE,3.0,09TH AVE 300,Control,Inner Richmond,NaN,01-jul-2012 08:00:00,93600.0,85690.0,...,NaN,0.915491,0.084509,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
sensor = sensor.drop(['TOTAL_TIME','TOTAL_OCCUPIED_TIME', 'TOTAL_VACANT_TIME', 'TOTAL_UNKNOWN_TIME',
       'OP_TIME', 'OP_OCCUPIED_TIME', 'OP_VACANT_TIME', 'OP_UNKNOWN_TIME',
       'NONOP_TIME', 'NONOP_OCCUPIED_TIME', 'NONOP_VACANT_TIME',
       'NONOP_UNKNOWN_TIME', 'GMP_TIME', 'GMP_OCCUPIED_TIME',
       'GMP_VACANT_TIME', 'GMP_UNKNOWN_TIME', 'COMM_TIME',
       'COMM_OCCUPIED_TIME', 'COMM_VACANT_TIME', 'COMM_UNKNOWN_TIME'], axis = 1)

In [15]:
sensor = sensor.drop(['BLOCK_NUM','RATE', 'START_TIME_DT','CAL_DATE'], axis = 1)

In [33]:
summary = sensor.groupby(['STREET_NAME', 'DOW','TIME_OF_DAY'])[['TOTAL_OCCUPIED_TIME_pct',
       'TOTAL_VACANT_TIME_pct', 'TOTAL_UNKNOWN_TIME_pct',
       'OP_OCCUPIED_TIME_pct', 'OP_VACANT_TIME_pct', 'OP_UNKNOWN_TIME_pct',
       'NONOP_OCCUPIED_TIME_pct', 'NONOP_VACANT_TIME_pct',
       'NONOP_UNKNOWN_TIME_pct', 'GMP_OCCUPIED_TIME_pct',
       'GMP_VACANT_TIME_pct', 'GMP_UNKNOWN_TIME_pct', 'COMM_OCCUPIED_TIME_pct',
       'COMM_VACANT_TIME_pct', 'COMM_UNKNOWN_TIME_pct','TOTAL_TIME_spots','OP_TIME_spots', 'NONOP_TIME_spots',
       'GMP_TIME_spots', 'COMM_TIME_spots']].mean()

summary = summary.reset_index()

In [34]:
summary  = summary.fillna(0)
summary.head()

,STREET_NAME,DOW,TIME_OF_DAY,TOTAL_OCCUPIED_TIME_pct,TOTAL_VACANT_TIME_pct,TOTAL_UNKNOWN_TIME_pct,OP_OCCUPIED_TIME_pct,OP_VACANT_TIME_pct,OP_UNKNOWN_TIME_pct,NONOP_OCCUPIED_TIME_pct,...,GMP_VACANT_TIME_pct,GMP_UNKNOWN_TIME_pct,COMM_OCCUPIED_TIME_pct,COMM_VACANT_TIME_pct,COMM_UNKNOWN_TIME_pct,TOTAL_TIME_spots,OP_TIME_spots,NONOP_TIME_spots,GMP_TIME_spots,COMM_TIME_spots
0,01ST ST,0,0.0,0.354069,0.565198,0.080734,0.0,0.0,0.0,0.354069,...,0.0,0.0,0.0,0.0,0.0,35.423497,0.0,35.423497,0.0,0.0
1,01ST ST,0,100.0,0.342599,0.574450,0.082952,0.0,0.0,0.0,0.342599,...,0.0,0.0,0.0,0.0,0.0,35.423497,0.0,35.423497,0.0,0.0
2,01ST ST,0,200.0,0.334373,0.580756,0.084872,0.0,0.0,0.0,0.334373,...,0.0,0.0,0.0,0.0,0.0,35.423497,0.0,35.423497,0.0,0.0
3,01ST ST,0,300.0,0.330584,0.583502,0.085914,0.0,0.0,0.0,0.330584,...,0.0,0.0,0.0,0.0,0.0,35.423497,0.0,35.423497,0.0,0.0
4,01ST ST,0,400.0,0.335564,0.577616,0.086820,0.0,0.0,0.0,0.335564,...,0.0,0.0,0.0,0.0,0.0,35.423497,0.0,35.423497,0.0,0.0


In [38]:
cols = ['OP_TIME_spots', 'NONOP_TIME_spots','GMP_TIME_spots', 'COMM_TIME_spots']
for col in cols:
    summary[col] = summary[col].astype('int')

In [40]:
summary.to_csv('sensor_2013.csv')

In [41]:
summary

,STREET_NAME,DOW,TIME_OF_DAY,TOTAL_OCCUPIED_TIME_pct,TOTAL_VACANT_TIME_pct,TOTAL_UNKNOWN_TIME_pct,OP_OCCUPIED_TIME_pct,OP_VACANT_TIME_pct,OP_UNKNOWN_TIME_pct,NONOP_OCCUPIED_TIME_pct,...,GMP_VACANT_TIME_pct,GMP_UNKNOWN_TIME_pct,COMM_OCCUPIED_TIME_pct,COMM_VACANT_TIME_pct,COMM_UNKNOWN_TIME_pct,TOTAL_TIME_spots,OP_TIME_spots,NONOP_TIME_spots,GMP_TIME_spots,COMM_TIME_spots
0,01ST ST,0,0.0,0.354069,0.565198,0.080734,0.000000,0.000000,0.000000,0.354069,...,0.000000,0.000000,0.000000,0.000000,0.000000,35.423497,0,35,0,0
1,01ST ST,0,100.0,0.342599,0.574450,0.082952,0.000000,0.000000,0.000000,0.342599,...,0.000000,0.000000,0.000000,0.000000,0.000000,35.423497,0,35,0,0
2,01ST ST,0,200.0,0.334373,0.580756,0.084872,0.000000,0.000000,0.000000,0.334373,...,0.000000,0.000000,0.000000,0.000000,0.000000,35.423497,0,35,0,0
3,01ST ST,0,300.0,0.330584,0.583502,0.085914,0.000000,0.000000,0.000000,0.330584,...,0.000000,0.000000,0.000000,0.000000,0.000000,35.423497,0,35,0,0
4,01ST ST,0,400.0,0.335564,0.577616,0.086820,0.000000,0.000000,0.000000,0.335564,...,0.000000,0.000000,0.000000,0.000000,0.000000,35.423497,0,35,0,0
5,01ST ST,0,500.0,0.361245,0.550669,0.088086,0.000000,0.000000,0.000000,0.361245,...,0.000000,0.000000,0.000000,0.000000,0.000000,35.423497,0,35,0,0
6,01ST ST,0,600.0,0.415021,0.497059,0.087920,0.000000,0.000000,0.000000,0.415021,...,0.000000,0.000000,0.000000,0.000000,0.000000,35.423497,0,35,0,0
7,01ST ST,0,700.0,0.412081,0.363875,0.224043,0.376942,0.558877,0.064181,0.339510,...,0.545652,0.033805,0.378752,0.547597,0.073652,35.423497,19,15,11,8
8,01ST ST,0,800.0,0.455680,0.323909,0.220411,0.443223,0.496166,0.060611,0.354602,...,0.424832,0.032729,0.412302,0.517283,0.070415,35.423497,19,15,11,8
9,01ST ST,0,900.0,0.529176,0.395254,0.075570,0.532763,0.391753,0.075485,0.414601,...,0.279821,0.104896,0.414837,0.518125,0.067038,35.426230,33,2,20,13
